# Project 1

An Agent-Based SEIR Model

- Robert Morain
- Vigynesh Bhatt

### Abstract

Summarize what you did and what you learned

### Introduction

Short summary of why the problem is interesting to you (if it’s not interesting to
you, make something up), what you did, and how you organized your report.

### Experiment Conditions

I specified most of the experiment conditions, but I asked you to do
two things: first, choose parameters for your scale-free networks and justify those choices, and
second, run some experiments where you choose the infectiousness levels, time spent infectious,
and time spent exposed.

### Hypotheses 

Make some hypotheses about how the virus will propagate across the different
networks under the different conditions. Give some justification for your hypotheses in terms of
network characteristics (e.g., the metrics in your summary table).

### Discussion

Summarize which hypotheses were supported by data and which were not supported.
Explain why you think the results came out the way they did. If you are speculating about why,
state that you are hypothesizing a possible explanation. If the reason why is justified by the
data, tell me how the data supports your explanation.

### Future Work

Tell me what you wish you had done or could do now that the project is over.

In [1]:
import numpy as np
import networkx as nx

In [2]:
def p_infectious(d, p_1c, B):
    a = p_1c / (1 - p_1c)
    b = a * np.exp(B * (d ** 3 - 1))
    return (a * b) / (1 + a * b) 

In [3]:
m_e = 1.0
s_e = 1.0
m_i = 2.25
s_i = 0.105
B = -0.0050367
p_1c = 0.12

In [4]:
from enum import Enum

class State(Enum):
    SUSCEPTIBLE = 'S'
    EXPOSED = 'E'
    INFECTED = 'I'
    RECOVERED = 'R'   


In [5]:
class Agent():
    def __init__(self, m_e, s_e, m_i, s_i, neighbours, initial_state=State.SUSCEPTIBLE):
        self.state = initial_state
        self.neighbours = neighbours

        self.days_spent_infectious = 0 # How many days the agent has been infectious
        
        self.d_e = np.ceil(np.random.lognormal(mean=m_e, sigma=s_e)) # How long the agent stays in the exposed stage
        self.countdown_to_infectious = self.d_e
        self.d_i = np.ceil(np.random.lognormal(mean=m_i, sigma=s_i)) # How long the agent stays in the infectious stage
        self.countdown_to_recovered = self.d_i
    
    def setState(self, state):
        self.state = state

    def getState(self):
        return self.state

    def step(self):
        if self.state == State.SUSCEPTIBLE:
            self.do_susceptible()
        elif self.state == State.EXPOSED:
            self.do_exposed()
        elif self.state == State.INFECTED:
            self.do_infected()
        else:
            self.do_recovered()

    def do_susceptible(self):
        for neighbor in self.neighbors:
            if neighbor.state == State.INFECTED:
                self.setState(State.EXPOSED)
                return
    
    def do_exposed(self):
        self.countdown_to_infectious-=1
        if self.countdown_to_infectious == 0:
            self.setState(State.INFECTED)
    
    def do_infected(self):
        self.countdown_to_recovered-=1
        self.days_spent_infectious+=1
        if self.countdown_to_recovered==0:
            self.setState(State.RECOVERED)
    
    def do_recovered(self):
        pass



In [20]:
class Population():
    def __init__(self, G, m_e=1.0, s_e=1.0, m_i=2.25, s_i=0.105):
        N = len(G.nodes)
        self.population = []
        for i in range(N):
            neighbors = G[i]
            self.population.append(Agent(m_e, s_e, m_i, s_i, neighbors))
        # Set 5% of the population to exposed and 5% to infectious
        exposed_infectious = np.random.choice(self.population, size=int(N * 0.1), replace=False)
        for i in range(len(exposed_infectious) // 2):
            exposed_infectious[i].state = State.EXPOSED
        for i in range(len(exposed_infectious) // 2, len(exposed_infectious)):
            exposed_infectious[i].state = State.INFECTED
            
    def count_all(self):
        susceptible = 0
        exposed = 0
        infectious = 0
        recovered = 0
        for a in self.population:
            if a.state == State.SUSCEPTIBLE:
                susceptible += 1
            elif a.state == State.EXPOSED:
                exposed += 1
            elif a.state == State.INFECTED:
                infectious += 1
            elif a.state == State.RECOVERED:
                recovered += 1
        return susceptible, exposed, infectious, recovered
                
    
    def step_all(self):
        for a in self.all:
            a.step()

In [7]:
def _read_graph_from_file(filename="ia-infect-dublin/ia-infect-dublin.mtx"):
    fo = open(filename,"r")
    line = fo.readline() # Read file header
    line = fo.readline() # Number of vertices and edges
    if not line:
        print("error -- illegal format for input")
        return
    v = line.split(" ")
    numVertices = int(v[0])
    G = nx.Graph()
    G.add_nodes_from(range(1, numVertices+1))
    while True:
        line = fo.readline()
        if not line:
            break
        #print("Line{}: {}".format(count,line.strip()))
        v = line.split(" ")
        v1 = int(v[0])
        v2 = int(v[1])
        G.add_edge(v1,v2)
        G.add_edge(v2,v1)
    fo.close()
    return G

In [21]:
G = nx.complete_graph(100)

In [22]:
P = Population(G)

In [23]:
P.count_all()

(90, 5, 5, 0)

In [24]:
a = P.population[0]

In [28]:
a.neighbours

AtlasView({1: {}, 2: {}, 3: {}, 4: {}, 5: {}, 6: {}, 7: {}, 8: {}, 9: {}, 10: {}, 11: {}, 12: {}, 13: {}, 14: {}, 15: {}, 16: {}, 17: {}, 18: {}, 19: {}, 20: {}, 21: {}, 22: {}, 23: {}, 24: {}, 25: {}, 26: {}, 27: {}, 28: {}, 29: {}, 30: {}, 31: {}, 32: {}, 33: {}, 34: {}, 35: {}, 36: {}, 37: {}, 38: {}, 39: {}, 40: {}, 41: {}, 42: {}, 43: {}, 44: {}, 45: {}, 46: {}, 47: {}, 48: {}, 49: {}, 50: {}, 51: {}, 52: {}, 53: {}, 54: {}, 55: {}, 56: {}, 57: {}, 58: {}, 59: {}, 60: {}, 61: {}, 62: {}, 63: {}, 64: {}, 65: {}, 66: {}, 67: {}, 68: {}, 69: {}, 70: {}, 71: {}, 72: {}, 73: {}, 74: {}, 75: {}, 76: {}, 77: {}, 78: {}, 79: {}, 80: {}, 81: {}, 82: {}, 83: {}, 84: {}, 85: {}, 86: {}, 87: {}, 88: {}, 89: {}, 90: {}, 91: {}, 92: {}, 93: {}, 94: {}, 95: {}, 96: {}, 97: {}, 98: {}, 99: {}})

In [41]:
n = G.neighbors(0)

In [45]:
[n for n in G[0]]

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]